# 🤖 Agentes Autónomos para Automatización

Objetivo: construir agentes con LangGraph y AutoGen que automaticen tareas complejas de ingeniería de datos: debugging, optimización de queries, orquestación de pipelines.

- Duración: 120-150 min
- Dificultad: Alta
- Stack: LangChain, LangGraph, AutoGen

## 🧠 Agentes Autónomos: De ReAct a Multi-Agent Systems

Los **agentes autónomos** son sistemas que combinan LLMs con herramientas (tools) para realizar tareas complejas de forma iterativa, tomando decisiones basadas en observaciones y razonamiento. En Data Engineering, permiten automatizar debugging, optimización de queries, monitoreo de pipelines y respuesta a incidentes.

### 🏗️ Evolución de Agentes

```
2021: ReAct (Reasoning + Acting)      → LLM decide qué tool usar en cada paso
2022: MRKL Systems                     → Multi-tool reasoning con knowledge bases
2023: LangChain Agents                 → Framework con 100+ tools pre-built
2023: AutoGen (Microsoft)              → Multi-agent collaboration
2024: LangGraph                        → Workflows con estados y ciclos
2024: Agentes de código (Devin, etc.)  → Autonomous software engineering
```

### 📐 Arquitectura de Agentes ReAct

```
┌─────────────────────────────────────────────────────────────────┐
│                    AGENT REACT LOOP                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Input: "¿Cuántas filas tiene la tabla ventas y cuál es su      │
│          esquema?"                                               │
│        ↓                                                         │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 1. THOUGHT (Razonamiento)                             │     │
│  │    LLM analiza la pregunta y decide acción            │     │
│  │    "Necesito dos cosas: row count y schema.           │     │
│  │     Primero obtendré el row count."                   │     │
│  └────────────────────────────────────────────────────────┘     │
│        ↓                                                         │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 2. ACTION (Acción)                                     │     │
│  │    LLM selecciona tool y parámetros                    │     │
│  │    Tool: get_table_row_count                           │     │
│  │    Input: {"table_name": "ventas"}                     │     │
│  └────────────────────────────────────────────────────────┘     │
│        ↓                                                         │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 3. OBSERVATION (Resultado)                             │     │
│  │    Tool ejecuta y retorna resultado                    │     │
│  │    Output: "La tabla ventas tiene 1,250,000 filas."   │     │
│  └────────────────────────────────────────────────────────┘     │
│        ↓                                                         │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 4. THOUGHT (Re-evaluar)                                │     │
│  │    "Tengo el row count. Ahora necesito el schema."    │     │
│  └────────────────────────────────────────────────────────┘     │
│        ↓                                                         │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 5. ACTION                                              │     │
│  │    Tool: get_table_schema                              │     │
│  │    Input: {"table_name": "ventas"}                     │     │
│  └────────────────────────────────────────────────────────┘     │
│        ↓                                                         │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 6. OBSERVATION                                         │     │
│  │    Output: "venta_id (BIGINT), fecha (DATE), ..."     │     │
│  └────────────────────────────────────────────────────────┘     │
│        ↓                                                         │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 7. THOUGHT (Finalizar)                                 │     │
│  │    "Tengo toda la información necesaria."              │     │
│  └────────────────────────────────────────────────────────┘     │
│        ↓                                                         │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 8. FINAL ANSWER                                        │     │
│  │    "La tabla ventas tiene 1.25M filas. El esquema     │     │
│  │     incluye: venta_id (BIGINT), fecha (DATE)..."      │     │
│  └────────────────────────────────────────────────────────┘     │
│                                                                  │
│  Ciclo completo: 2 iteraciones, 2 tools, respuesta completa     │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### 🔧 Implementación de Agente para Data Engineering

```python
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from typing import Dict, List
import json

# LLM
llm = ChatOpenAI(model='gpt-4o', temperature=0, api_key=os.getenv('OPENAI_API_KEY'))

# Tools para Data Engineering
@tool
def get_table_metadata(table_name: str) -> str:
    """
    Obtiene metadatos completos de una tabla: row count, schema, particiones, tamaño.
    
    Args:
        table_name: nombre de la tabla (ej. 'dwh.ventas')
    
    Returns:
        JSON con metadatos
    """
    # En producción: conectar a Snowflake/BigQuery/Redshift
    metadata_db = {
        'dwh.ventas': {
            'row_count': 125_000_000,
            'size_gb': 45.2,
            'schema': {
                'venta_id': 'BIGINT PRIMARY KEY',
                'fecha': 'DATE NOT NULL',
                'cliente_id': 'INT FOREIGN KEY',
                'producto_id': 'INT FOREIGN KEY',
                'cantidad': 'INT',
                'total': 'DECIMAL(10,2)'
            },
            'partitions': ['fecha'],
            'last_updated': '2024-10-30 23:45:00',
            'owner': 'data-engineering'
        },
        'dwh.clientes': {
            'row_count': 5_000_000,
            'size_gb': 2.8,
            'schema': {
                'cliente_id': 'INT PRIMARY KEY',
                'nombre': 'VARCHAR(100)',
                'email': 'VARCHAR(100)',
                'ciudad': 'VARCHAR(50)',
                'fecha_registro': 'DATE'
            },
            'partitions': None,
            'last_updated': '2024-10-30 22:00:00',
            'owner': 'data-engineering'
        }
    }
    
    if table_name not in metadata_db:
        return json.dumps({'error': f'Tabla {table_name} no encontrada'})
    
    return json.dumps(metadata_db[table_name], indent=2)

@tool
def explain_query_plan(query: str) -> str:
    """
    Analiza el execution plan de una query SQL y identifica cuellos de botella.
    
    Args:
        query: query SQL a analizar
    
    Returns:
        Análisis del plan de ejecución con recomendaciones
    """
    # En producción: EXPLAIN ANALYZE en la base de datos
    analysis = {
        'estimated_rows': 1_250_000,
        'estimated_cost': 15234.50,
        'operations': [
            {
                'step': 1,
                'operation': 'Seq Scan on ventas',
                'cost': 12000.00,
                'rows': 125_000_000,
                'warning': 'Full table scan - considera agregar índice'
            },
            {
                'step': 2,
                'operation': 'Hash Join',
                'cost': 3234.50,
                'rows': 1_250_000,
                'info': 'Join eficiente con hash'
            }
        ],
        'recommendations': [
            'Agregar índice en ventas.fecha para filtros',
            'Considerar particionamiento por fecha',
            'Usar SELECT con columnas específicas en vez de *'
        ]
    }
    
    return json.dumps(analysis, indent=2)

@tool
def check_pipeline_status(pipeline_name: str) -> str:
    """
    Verifica el estado de un pipeline de Airflow/Prefect.
    
    Args:
        pipeline_name: nombre del DAG/flow
    
    Returns:
        Estado actual y últimas ejecuciones
    """
    # En producción: conectar a Airflow API
    pipeline_status = {
        'daily_sales_etl': {
            'status': 'FAILED',
            'last_run': '2024-10-30 23:00:00',
            'duration_seconds': 1245,
            'error': 'Connection timeout to Snowflake',
            'recent_runs': [
                {'date': '2024-10-30', 'status': 'FAILED', 'duration': 1245},
                {'date': '2024-10-29', 'status': 'SUCCESS', 'duration': 892},
                {'date': '2024-10-28', 'status': 'SUCCESS', 'duration': 901}
            ],
            'next_run': '2024-10-31 23:00:00'
        }
    }
    
    if pipeline_name not in pipeline_status:
        return json.dumps({'error': f'Pipeline {pipeline_name} no encontrado'})
    
    return json.dumps(pipeline_status[pipeline_name], indent=2)

@tool
def run_data_quality_check(table_name: str, check_type: str) -> str:
    """
    Ejecuta validaciones de calidad de datos.
    
    Args:
        table_name: tabla a validar
        check_type: tipo de check ('nulls', 'duplicates', 'freshness', 'schema')
    
    Returns:
        Resultados de la validación
    """
    # En producción: integrar con Great Expectations, dbt tests, etc.
    quality_results = {
        'dwh.ventas': {
            'nulls': {
                'total': 'NULL rate: 0.05%',
                'fecha': 'NULL rate: 0%',
                'cliente_id': 'NULL rate: 0.2%',
                'status': 'PASS'
            },
            'duplicates': {
                'count': 120,
                'percentage': '0.0001%',
                'status': 'WARNING'
            },
            'freshness': {
                'last_update': '2024-10-30 23:45:00',
                'delay_hours': 0.25,
                'status': 'PASS'
            },
            'schema': {
                'columns_expected': 6,
                'columns_actual': 6,
                'mismatches': [],
                'status': 'PASS'
            }
        }
    }
    
    if table_name not in quality_results:
        return json.dumps({'error': f'No hay checks configurados para {table_name}'})
    
    if check_type not in quality_results[table_name]:
        return json.dumps({'error': f'Check type {check_type} no soportado'})
    
    return json.dumps(quality_results[table_name][check_type], indent=2)

@tool
def suggest_query_optimization(query: str) -> str:
    """
    Analiza una query y sugiere optimizaciones específicas.
    
    Args:
        query: query SQL a optimizar
    
    Returns:
        Lista de sugerencias con query mejorada
    """
    suggestions = []
    optimized_query = query
    
    # Análisis de patrones anti-pattern
    if 'SELECT *' in query.upper():
        suggestions.append({
            'issue': 'SELECT * es ineficiente',
            'impact': 'Alto - lee columnas innecesarias',
            'recommendation': 'Especificar solo columnas necesarias',
            'example': 'SELECT id, fecha, total FROM ...'
        })
    
    if 'WHERE' not in query.upper() and 'JOIN' in query.upper():
        suggestions.append({
            'issue': 'JOIN sin filtros WHERE',
            'impact': 'Alto - procesa todos los registros',
            'recommendation': 'Agregar filtros WHERE para reducir dataset',
            'example': 'WHERE fecha >= CURRENT_DATE - 30'
        })
    
    if 'DISTINCT' in query.upper() and 'GROUP BY' not in query.upper():
        suggestions.append({
            'issue': 'DISTINCT sin GROUP BY puede ser lento',
            'impact': 'Medio - sort costoso',
            'recommendation': 'Considerar GROUP BY si es posible',
            'example': 'GROUP BY columnas en vez de DISTINCT'
        })
    
    return json.dumps({
        'original_query': query,
        'suggestions': suggestions,
        'priority': 'HIGH' if len(suggestions) >= 2 else 'MEDIUM'
    }, indent=2)

# Lista de tools
tools = [
    get_table_metadata,
    explain_query_plan,
    check_pipeline_status,
    run_data_quality_check,
    suggest_query_optimization
]

# Prompt customizado para Data Engineering
prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un Senior Data Engineer experto en:
- Análisis y optimización de queries SQL
- Debugging de pipelines de datos (Airflow, dbt, Spark)
- Data quality y governance
- Arquitectura de data warehouses (Snowflake, BigQuery, Redshift)

Tu trabajo es ayudar a resolver problemas usando las tools disponibles.
Siempre razona paso a paso y usa múltiples tools si es necesario para dar una respuesta completa.

Cuando analices problemas:
1. Primero obtén contexto (metadatos, estado actual)
2. Identifica el root cause
3. Sugiere soluciones accionables
4. Prioriza por impacto

Sé conciso pero preciso."""),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

# Crear agente
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    max_iterations=10,  # Límite de seguridad
    max_execution_time=60,  # Timeout 60 segundos
    handle_parsing_errors=True  # Manejo robusto de errores
)

# Ejemplo 1: Investigación de tabla
print("=" * 80)
print("CASO 1: Análisis de tabla")
print("=" * 80)

result1 = agent_executor.invoke({
    "input": """Necesito analizar la tabla dwh.ventas.
    ¿Cuántas filas tiene? ¿Está bien particionada? ¿Cuál es su tamaño?"""
})

print("\n📊 Respuesta del agente:")
print(result1['output'])

# Ejemplo 2: Debugging de pipeline
print("\n" + "=" * 80)
print("CASO 2: Pipeline fallando")
print("=" * 80)

result2 = agent_executor.invoke({
    "input": """El pipeline 'daily_sales_etl' está fallando.
    Investiga qué pasó y sugiere cómo solucionarlo."""
})

print("\n🔧 Respuesta del agente:")
print(result2['output'])

# Ejemplo 3: Optimización de query
print("\n" + "=" * 80)
print("CASO 3: Query lenta")
print("=" * 80)

slow_query = """
SELECT * 
FROM dwh.ventas v
JOIN dwh.clientes c ON v.cliente_id = c.cliente_id
"""

result3 = agent_executor.invoke({
    "input": f"""Esta query es muy lenta: {slow_query}
    Analiza el execution plan y sugiere optimizaciones."""
})

print("\n⚡ Respuesta del agente:")
print(result3['output'])
```

### 📊 Comparación de Frameworks de Agentes

| Framework | Tipo | Complejidad | Casos de Uso | Estado |
|-----------|------|-------------|--------------|---------|
| **LangChain Agents** | Single agent + tools | Medio | Tareas simples con 1-5 tools | Estable, producción |
| **LangGraph** | Stateful workflows | Alto | Workflows complejos, ciclos, branches | Estable, recomendado |
| **AutoGen** | Multi-agent collaboration | Muy alto | Múltiples agentes especializados | Experimental |
| **CrewAI** | Hierarchical multi-agent | Alto | Equipos con roles y jerarquías | Emergente |
| **BabyAGI / AutoGPT** | Autonomous goal-oriented | Muy alto | Investigación, no producción | Experimental |

### 🎯 Anatomía de una Tool de Calidad

```python
from pydantic import BaseModel, Field
from typing import Literal

class QueryOptimizationInput(BaseModel):
    """Schema de input para validación."""
    query: str = Field(description="Query SQL a optimizar")
    database: Literal["snowflake", "bigquery", "redshift"] = Field(
        default="snowflake",
        description="Tipo de base de datos"
    )
    max_cost: float = Field(
        default=1000.0,
        description="Costo máximo aceptable del query"
    )

@tool(args_schema=QueryOptimizationInput)
def optimize_query_production(query: str, database: str = "snowflake", max_cost: float = 1000.0) -> str:
    """
    Tool de producción para optimización de queries con validación.
    
    Características:
    - Type hints y validation con Pydantic
    - Error handling robusto
    - Logging de todas las invocaciones
    - Timeouts y rate limiting
    - Retries con exponential backoff
    """
    import logging
    import time
    from tenacity import retry, stop_after_attempt, wait_exponential
    
    logger = logging.getLogger(__name__)
    logger.info(f"optimize_query invoked: database={database}, max_cost={max_cost}")
    
    try:
        # Validación de input
        if len(query) > 10000:
            raise ValueError("Query demasiado larga (>10K caracteres)")
        
        if not query.strip().upper().startswith('SELECT'):
            raise ValueError("Solo queries SELECT están soportadas")
        
        # Simulación de análisis con retry
        @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
        def analyze_with_retry():
            # En producción: llamar a servicio de análisis
            time.sleep(0.5)  # Simular latencia
            return {
                'estimated_cost': 850.0,
                'suggestions': ['Agregar índice en fecha', 'Usar columnas específicas'],
                'optimized_query': query.replace('SELECT *', 'SELECT id, fecha')
            }
        
        result = analyze_with_retry()
        
        # Validación de output
        if result['estimated_cost'] > max_cost:
            logger.warning(f"Query costo {result['estimated_cost']} excede máximo {max_cost}")
            return json.dumps({
                'status': 'WARNING',
                'message': f'Costo estimado {result["estimated_cost"]} excede límite {max_cost}',
                'suggestions': result['suggestions']
            })
        
        logger.info(f"Optimización exitosa: costo reducido a {result['estimated_cost']}")
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"Error en optimize_query: {str(e)}")
        return json.dumps({'error': str(e), 'status': 'FAILED'})
```

### 🚨 Patrones de Seguridad y Control

```python
from typing import Callable
import functools

def require_approval(func: Callable) -> Callable:
    """Decorator para tools que requieren aprobación humana."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"\n⚠️  ACCIÓN REQUIERE APROBACIÓN:")
        print(f"   Tool: {func.__name__}")
        print(f"   Args: {args}, {kwargs}")
        
        approval = input("   ¿Aprobar? (y/n): ")
        if approval.lower() != 'y':
            return json.dumps({'status': 'REJECTED', 'reason': 'User declined'})
        
        return func(*args, **kwargs)
    return wrapper

@tool
@require_approval
def drop_table(table_name: str) -> str:
    """
    Elimina una tabla (PELIGROSO - requiere aprobación).
    
    Args:
        table_name: nombre de la tabla a eliminar
    """
    # En producción: ejecutar DROP TABLE
    return json.dumps({
        'status': 'SUCCESS',
        'message': f'Tabla {table_name} eliminada',
        'timestamp': '2024-10-30 23:50:00'
    })

# Rate limiting para evitar costos excesivos
from collections import deque
import time

class RateLimiter:
    """Rate limiter para tools costosas."""
    
    def __init__(self, max_calls: int, time_window: int):
        self.max_calls = max_calls
        self.time_window = time_window
        self.calls = deque()
    
    def allow_call(self) -> bool:
        now = time.time()
        
        # Remover llamadas fuera de la ventana
        while self.calls and self.calls[0] < now - self.time_window:
            self.calls.popleft()
        
        if len(self.calls) >= self.max_calls:
            return False
        
        self.calls.append(now)
        return True

# Uso
limiter = RateLimiter(max_calls=10, time_window=60)  # 10 calls/minuto

@tool
def expensive_llm_analysis(text: str) -> str:
    """Tool costosa con rate limiting."""
    if not limiter.allow_call():
        return json.dumps({
            'error': 'Rate limit exceeded',
            'message': 'Máximo 10 llamadas por minuto'
        })
    
    # Análisis costoso...
    return json.dumps({'result': 'analysis complete'})
```

### 💰 Optimización de Costos

```python
# Estrategia 1: Usar modelos pequeños para decisiones simples
llm_cheap = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)  # $0.50/$1.50 por 1M tokens
llm_expensive = ChatOpenAI(model='gpt-4o', temperature=0)     # $2.50/$10.00 por 1M tokens

# Estrategia 2: Limitar iteraciones
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    max_iterations=5,  # Evitar loops infinitos
    early_stopping_method="generate"  # Forzar respuesta después de max_iterations
)

# Estrategia 3: Cache de decisiones
from functools import lru_cache

@lru_cache(maxsize=1000)
def cached_tool_call(tool_name: str, input_hash: str):
    """Cache de resultados de tools determinísticas."""
    # Si el tool siempre retorna lo mismo para el mismo input, cachear
    pass

# Estrategia 4: Monitoring de costos
import tiktoken

def estimate_agent_cost(messages: List[dict], model: str = "gpt-4o") -> float:
    """Estima costo de una conversación de agente."""
    encoding = tiktoken.encoding_for_model(model)
    
    total_tokens = sum(len(encoding.encode(msg['content'])) for msg in messages)
    
    # Precios por 1M tokens (input / output)
    prices = {
        'gpt-4o': (2.50, 10.00),
        'gpt-3.5-turbo': (0.50, 1.50)
    }
    
    # Asumir 50% input, 50% output
    input_tokens = total_tokens * 0.5
    output_tokens = total_tokens * 0.5
    
    input_price, output_price = prices[model]
    cost = (input_tokens / 1_000_000 * input_price) + (output_tokens / 1_000_000 * output_price)
    
    return cost

# Ejemplo
messages = [
    {'role': 'system', 'content': 'Eres un data engineer...'},
    {'role': 'user', 'content': 'Analiza esta tabla...'},
    {'role': 'assistant', 'content': 'Voy a obtener los metadatos...'}
]

cost = estimate_agent_cost(messages, model='gpt-4o')
print(f"Costo estimado: ${cost:.4f}")
```

### 🎯 Reglas de Oro para Agentes en Producción

1. **Siempre validar inputs**: Tools reciben texto generado por LLM (puede ser malicioso)
2. **Timeouts obligatorios**: Max execution time para evitar loops infinitos
3. **Human-in-the-loop**: Acciones destructivas (DROP, DELETE, TRUNCATE) requieren aprobación
4. **Logging exhaustivo**: Toda decisión del agente debe loggearse para debugging
5. **Rate limiting**: Proteger APIs externas y controlar costos
6. **Graceful degradation**: Si tool falla, el agente debe continuar con otras tools
7. **Monitoring de costos**: Alertar si costo por sesión excede threshold
8. **Testing con mocks**: Probar agentes sin ejecutar tools reales

## 🔄 LangGraph: Workflows Complejos con Estados y Ciclos

**LangGraph** es un framework para construir agentes con **flujos de trabajo complejos** que incluyen estados persistentes, ramificaciones condicionales, ciclos (loops) y múltiples rutas de ejecución. Ideal para pipelines de Data Engineering donde las decisiones dependen de resultados previos.

### 🏗️ Arquitectura de LangGraph

```
┌─────────────────────────────────────────────────────────────────┐
│               LANGGRAPH: STATEFUL WORKFLOW                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Componentes Principales:                                        │
│                                                                  │
│  1️⃣ STATE: Objeto compartido entre todos los nodos             │
│     ┌───────────────────────────────────────────┐               │
│     │ class DataPipelineState(TypedDict):      │               │
│     │     query: str                            │               │
│     │     validated: bool                       │               │
│     │     optimized: bool                       │               │
│     │     execution_plan: dict                  │               │
│     │     results: list                         │               │
│     │     errors: list                          │               │
│     └───────────────────────────────────────────┘               │
│                                                                  │
│  2️⃣ NODES: Funciones que modifican el estado                   │
│     def validate_node(state) -> state:                          │
│         state['validated'] = check_syntax(state['query'])       │
│         return state                                            │
│                                                                  │
│  3️⃣ EDGES: Conexiones entre nodos (condicionales o fijas)      │
│     - Simple edge: validate → optimize                          │
│     - Conditional edge: if validated → optimize else → error    │
│                                                                  │
│  4️⃣ GRAPH: Estructura del workflow                             │
│                                                                  │
│                    START                                         │
│                      │                                           │
│                      ▼                                           │
│              [Validate Query]                                    │
│                      │                                           │
│                      ▼                                           │
│                  ┌───────┐                                       │
│                  │ Valid? │                                      │
│                  └───┬───┘                                       │
│                      │                                           │
│            ┌─────────┴─────────┐                                │
│            │                   │                                │
│          Yes                  No                                │
│            │                   │                                │
│            ▼                   ▼                                │
│       [Optimize]        [Error Handler]                         │
│            │                   │                                │
│            ▼                   ▼                                │
│        [Execute]           [Report]                             │
│            │                   │                                │
│            ▼                   ▼                                │
│        [Monitor]              END                               │
│            │                                                     │
│            ▼                                                     │
│        ┌────────┐                                               │
│        │Success?│                                               │
│        └───┬────┘                                               │
│            │                                                     │
│      ┌─────┴─────┐                                              │
│      │           │                                              │
│     Yes         No                                              │
│      │           │                                              │
│      ▼           ▼                                              │
│     END      [Retry]                                            │
│                │                                                │
│                └─────────────┐                                  │
│                              │                                  │
│                          (loop back                             │
│                           to Execute)                           │
│                              │                                  │
│                              └──────────────────────────────┐   │
│                                                             │   │
│                              ┌──────────────────────────────┘   │
│                              │                                  │
│                              ▼                                  │
│                          [Execute]                              │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### 🔧 Implementación: Pipeline ETL con Validación y Retry

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Literal
import operator
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import time

# 1. Definir estado del workflow
class ETLPipelineState(TypedDict):
    """Estado compartido del pipeline ETL."""
    source_query: str
    validated: bool
    optimized: bool
    execution_plan: dict
    executed: bool
    results: Annotated[list, operator.add]  # operator.add permite append
    errors: Annotated[list, operator.add]
    retry_count: int
    max_retries: int

# 2. Definir nodos (funciones que transforman el estado)

def validate_query_node(state: ETLPipelineState) -> ETLPipelineState:
    """Valida sintaxis de la query."""
    print(f"\n🔍 VALIDATE: Validando query...")
    
    query = state['source_query']
    
    # Validaciones básicas
    errors = []
    if not query.strip().upper().startswith('SELECT'):
        errors.append('Query debe comenzar con SELECT')
    
    if query.count('(') != query.count(')'):
        errors.append('Paréntesis desbalanceados')
    
    if 'SELCT' in query.upper():
        errors.append('Typo detectado: SELCT → SELECT')
    
    # LLM para validación semántica avanzada
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)  # Modelo barato para validación
    
    messages = [
        SystemMessage(content="Eres un experto en SQL. Valida la query y reporta errores."),
        HumanMessage(content=f"Query: {query}\n\n¿Hay errores de sintaxis o lógica?")
    ]
    
    try:
        response = llm.invoke(messages)
        llm_feedback = response.content
        
        if 'error' in llm_feedback.lower() or 'incorrecto' in llm_feedback.lower():
            errors.append(f"LLM feedback: {llm_feedback}")
    except Exception as e:
        errors.append(f"Error al validar con LLM: {str(e)}")
    
    if errors:
        state['validated'] = False
        state['errors'].extend(errors)
        print(f"   ❌ Errores encontrados: {errors}")
    else:
        state['validated'] = True
        print(f"   ✅ Query válida")
    
    return state

def optimize_query_node(state: ETLPipelineState) -> ETLPipelineState:
    """Optimiza la query usando LLM."""
    print(f"\n⚡ OPTIMIZE: Optimizando query...")
    
    if not state['validated']:
        print("   ⚠️ Query no validada, skip optimization")
        return state
    
    llm = ChatOpenAI(model='gpt-4o', temperature=0.3)
    
    messages = [
        SystemMessage(content="Eres un experto en optimización de queries SQL."),
        HumanMessage(content=f"""
Optimiza esta query:

{state['source_query']}

Aplica:
- Índices sugeridos
- Reescritura más eficiente
- Evitar subqueries costosas
- Usar CTEs cuando sea apropiado

Retorna la query optimizada:
""")
    ]
    
    try:
        response = llm.invoke(messages)
        optimized_query = response.content.strip()
        state['optimized'] = True
        state['source_query'] = optimized_query  # Reemplazar con versión optimizada
        print(f"   ✅ Query optimizada")
    except Exception as e:
        state['errors'].append(f"Error al optimizar: {str(e)}")
        print(f"   ❌ Error: {str(e)}")
    
    return state

def execute_query_node(state: ETLPipelineState) -> ETLPipelineState:
    """Ejecuta la query (simulado)."""
    print(f"\n🚀 EXECUTE: Ejecutando query...")
    
    # Simular ejecución (en producción: ejecutar en DB real)
    time.sleep(2)
    
    # Simular resultados
    fake_results = [
        {'id': 1, 'producto': 'Laptop', 'ventas': 150000},
        {'id': 2, 'producto': 'Mouse', 'ventas': 45000},
        {'id': 3, 'producto': 'Teclado', 'ventas': 78000}
    ]
    
    state['executed'] = True
    state['results'].extend(fake_results)
    print(f"   ✅ Query ejecutada: {len(fake_results)} resultados")
    
    return state

def monitor_node(state: ETLPipelineState) -> ETLPipelineState:
    """Monitorea métricas de ejecución."""
    print(f"\n📊 MONITOR: Verificando métricas...")
    
    # Verificar si hay resultados
    if len(state['results']) == 0:
        state['errors'].append("No se obtuvieron resultados")
        print("   ❌ Sin resultados")
        return state
    
    # Verificar límites (ej. no más de 1M rows)
    if len(state['results']) > 1_000_000:
        state['errors'].append(f"Demasiados resultados: {len(state['results'])}")
        print(f"   ⚠️ Demasiados resultados")
        return state
    
    print(f"   ✅ Métricas OK: {len(state['results'])} rows")
    return state

def retry_node(state: ETLPipelineState) -> ETLPipelineState:
    """Incrementa contador de retry."""
    print(f"\n🔄 RETRY: Intento {state['retry_count'] + 1}/{state['max_retries']}...")
    
    state['retry_count'] += 1
    state['executed'] = False  # Reset para volver a ejecutar
    
    # Limpiar errores del intento anterior
    state['errors'] = []
    
    time.sleep(2)  # Backoff
    
    return state

def error_handler_node(state: ETLPipelineState) -> ETLPipelineState:
    """Maneja errores y genera reporte."""
    print(f"\n❌ ERROR HANDLER: Procesando errores...")
    
    errors_summary = "\n".join(f"  - {err}" for err in state['errors'])
    print(f"Errores detectados:\n{errors_summary}")
    
    return state

# 3. Definir funciones de routing (conditional edges)

def should_optimize(state: ETLPipelineState) -> Literal["optimize", "error"]:
    """Decide si optimizar o manejar error."""
    return "optimize" if state['validated'] else "error"

def should_retry(state: ETLPipelineState) -> Literal["retry", "end"]:
    """Decide si reintentar o terminar."""
    
    has_errors = len(state['errors']) > 0
    can_retry = state['retry_count'] < state['max_retries']
    
    if has_errors and can_retry:
        return "retry"
    else:
        return "end"

# 4. Construir el grafo

workflow = StateGraph(ETLPipelineState)

# Agregar nodos
workflow.add_node("validate", validate_query_node)
workflow.add_node("optimize", optimize_query_node)
workflow.add_node("execute", execute_query_node)
workflow.add_node("monitor", monitor_node)
workflow.add_node("retry", retry_node)
workflow.add_node("error_handler", error_handler_node)

# Definir edges
workflow.set_entry_point("validate")

# Conditional edge después de validación
workflow.add_conditional_edges(
    "validate",
    should_optimize,
    {
        "optimize": "optimize",
        "error": "error_handler"
    }
)

# Edge simple: optimize → execute
workflow.add_edge("optimize", "execute")

# Edge simple: execute → monitor
workflow.add_edge("execute", "monitor")

# Conditional edge después de monitor
workflow.add_conditional_edges(
    "monitor",
    should_retry,
    {
        "retry": "retry",
        "end": END
    }
)

# Edge para ciclo de retry: retry → execute
workflow.add_edge("retry", "execute")

# Edge para error handler → END
workflow.add_edge("error_handler", END)

# Compilar el grafo
app = workflow.compile()

# 5. Ejecutar el workflow

print("=" * 70)
print("EJECUTANDO WORKFLOW ETL CON LANGGRAPH")
print("=" * 70)

initial_state = {
    'source_query': '''
        SELECT p.producto, SUM(v.total) as ventas_totales
        FROM ventas v
        JOIN productos p ON v.producto_id = p.id
        WHERE v.fecha >= '2024-01-01'
        GROUP BY p.producto
        ORDER BY ventas_totales DESC
        LIMIT 10
    ''',
    'validated': False,
    'optimized': False,
    'execution_plan': {},
    'executed': False,
    'results': [],
    'errors': [],
    'retry_count': 0,
    'max_retries': 3
}

# Ejecutar workflow
final_state = app.invoke(initial_state)

print("\n" + "=" * 70)
print("RESULTADOS FINALES")
print("=" * 70)
print(f"✅ Validado: {final_state['validated']}")
print(f"✅ Optimizado: {final_state['optimized']}")
print(f"✅ Ejecutado: {final_state['executed']}")
print(f"📊 Resultados: {len(final_state['results'])} filas")
print(f"❌ Errores: {len(final_state['errors'])}")
print(f"🔄 Reintentos: {final_state['retry_count']}")

if final_state['results']:
    print("\nPrimeros 3 resultados:")
    for row in final_state['results'][:3]:
        print(f"  {row}")
```

### 📊 Visualización del Grafo

```python
# LangGraph permite visualizar el workflow
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"⚠️ No se puede visualizar (instalar graphviz): {e}")
    print("\nEstructura del grafo:")
    print(app.get_graph().to_json())
```

### 💾 Persistencia de Estado (Checkpointing)

LangGraph permite guardar el estado del workflow y recuperarlo después:

```python
from langgraph.checkpoint.memory import MemorySaver

# Crear grafo con checkpointing
memory = MemorySaver()
app_with_memory = workflow.compile(checkpointer=memory)

# Ejecutar con thread_id para identificar la sesión
config = {"configurable": {"thread_id": "pipeline-123"}}

# Primera ejecución
result1 = app_with_memory.invoke(initial_state, config)

# Puedes recuperar el estado más tarde
print("\nRecuperando estado guardado...")
history = app_with_memory.get_state_history(config)
for state in history:
    print(f"  Estado en paso {state.values.get('retry_count', 0)}")
```

---

### 📊 Comparación: LangChain Agent vs LangGraph

| Aspecto | LangChain Agent | LangGraph |
|---------|-----------------|-----------|
| **Arquitectura** | Loop simple (Thought → Action → Observation) | Grafo con estados y múltiples rutas |
| **Control de flujo** | Lineal con decisiones del LLM | Explícito con conditional edges |
| **Estado** | Memoria de conversación | Estado tipado persistente |
| **Ciclos** | No soporta (solo iteraciones) | Soporta ciclos explícitos |
| **Debugging** | Difícil (black box) | Fácil (cada nodo visible) |
| **Costos** | Alto (muchas llamadas LLM) | Menor (decisiones programáticas) |
| **Casos de uso** | Tareas exploratorias, Q&A | Workflows complejos, pipelines ETL |

---

### 🎯 Patrón: Map-Reduce con LangGraph

```python
from typing import List

class MapReduceState(TypedDict):
    """Estado para map-reduce."""
    input_data: List[str]
    mapped_results: Annotated[List[dict], operator.add]
    reduced_result: dict

def map_node(state: MapReduceState) -> MapReduceState:
    """Aplica transformación a cada elemento (paralelizable)."""
    print(f"\n🗺️ MAP: Procesando {len(state['input_data'])} elementos...")
    
    for item in state['input_data']:
        # Simular procesamiento paralelo
        result = {'item': item, 'length': len(item), 'uppercase': item.upper()}
        state['mapped_results'].append(result)
    
    return state

def reduce_node(state: MapReduceState) -> MapReduceState:
    """Agrega resultados del map."""
    print(f"\n📊 REDUCE: Agregando {len(state['mapped_results'])} resultados...")
    
    total_length = sum(r['length'] for r in state['mapped_results'])
    state['reduced_result'] = {
        'total_items': len(state['mapped_results']),
        'total_length': total_length,
        'avg_length': total_length / len(state['mapped_results'])
    }
    
    return state

# Construir workflow map-reduce
mr_workflow = StateGraph(MapReduceState)
mr_workflow.add_node("map", map_node)
mr_workflow.add_node("reduce", reduce_node)
mr_workflow.set_entry_point("map")
mr_workflow.add_edge("map", "reduce")
mr_workflow.add_edge("reduce", END)

mr_app = mr_workflow.compile()

# Ejecutar
mr_state = {
    'input_data': ['data engineering', 'machine learning', 'data science'],
    'mapped_results': [],
    'reduced_result': {}
}

final_mr_state = mr_app.invoke(mr_state)
print(f"\n✅ Resultado reducido: {final_mr_state['reduced_result']}")
```

---

### 💡 Mejores Prácticas con LangGraph

1. **🎯 Estado tipado:** Usa `TypedDict` para validar el estado
2. **🔍 Logging granular:** Cada nodo debe loggear su operación
3. **⚡ Decisiones programáticas:** Usa conditional edges en vez de LLM cuando sea posible (más rápido, más barato)
4. **🔄 Idempotencia:** Los nodos deben ser idempotentes (safe to retry)
5. **💾 Checkpointing:** Usa `MemorySaver` para pipelines largos
6. **📊 Monitoreo:** Agrega nodos de monitoring/alerting
7. **🧪 Testing:** Cada nodo debe ser testeable independientemente

## 👥 AutoGen: Sistemas Multi-Agente Colaborativos

**AutoGen** (Microsoft) permite crear sistemas donde **múltiples agentes especializados colaboran** para resolver problemas complejos. Cada agente tiene un rol, personalidad y objetivos propios, similar a un equipo humano trabajando juntos.

### 🏗️ Arquitectura Multi-Agente

```
┌─────────────────────────────────────────────────────────────────┐
│            AUTOGEN: MULTI-AGENT COLLABORATION                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Roles de Agentes:                                               │
│                                                                  │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 1. AssistantAgent                                      │     │
│  │    - LLM-powered agent que resuelve tareas            │     │
│  │    - Puede usar code execution                         │     │
│  │    - Responde a instrucciones                          │     │
│  └────────────────────────────────────────────────────────┘     │
│                                                                  │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 2. UserProxyAgent                                      │     │
│  │    - Representa al usuario humano                      │     │
│  │    - Ejecuta código en sandbox                         │     │
│  │    - Puede requerir aprobación humana                  │     │
│  └────────────────────────────────────────────────────────┘     │
│                                                                  │
│  ┌────────────────────────────────────────────────────────┐     │
│  │ 3. GroupChat                                           │     │
│  │    - Orquesta conversación entre N agentes            │     │
│  │    - Round-robin o speaker selection automático        │     │
│  │    - Termina cuando objetivo se cumple                 │     │
│  └────────────────────────────────────────────────────────┘     │
│                                                                  │
│  Ejemplo: Diseño de Pipeline ETL                                │
│                                                                  │
│     User                                                         │
│       │ "Diseña pipeline para ingestar datos de API Stripe"    │
│       ↓                                                          │
│  ┌─────────────┐                                                │
│  │ Data Engineer│ "Voy a diseñar un pipeline con:              │
│  │  (Assistant) │  1. Extract: API calls con retry             │
│  │              │  2. Transform: Pandas + validación            │
│  │              │  3. Load: Snowflake con upsert"              │
│  └──────┬───────┘                                                │
│         │                                                        │
│         ↓                                                        │
│  ┌─────────────┐                                                │
│  │ QA Engineer  │ "Detecto problemas:                           │
│  │ (Assistant)  │  - Falta manejo de rate limits                │
│  │              │  - No hay logging                             │
│  │              │  - ¿Qué pasa si API cambia schema?"          │
│  └──────┬───────┘                                                │
│         │                                                        │
│         ↓                                                        │
│  ┌─────────────┐                                                │
│  │ Data Engineer│ "Actualización:                               │
│  │              │  - Agregado: exponential backoff              │
│  │              │  - Agregado: structured logging               │
│  │              │  - Agregado: schema validation con pydantic"  │
│  └──────┬───────┘                                                │
│         │                                                        │
│         ↓                                                        │
│  ┌─────────────┐                                                │
│  │ QA Engineer  │ "Aprobado! Ahora agregar tests unitarios."   │
│  └──────┬───────┘                                                │
│         │                                                        │
│         ↓                                                        │
│  ┌─────────────┐                                                │
│  │ User Proxy   │ Ejecuta código generado en sandbox            │
│  │              │ "Tests pasan ✅"                              │
│  └─────────────┘                                                │
│                                                                  │
│  Resultado: Pipeline completo diseñado, revisado, y testeado    │
│             en 4-5 iteraciones de conversación                   │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### 🔧 Implementación: Team de Data Engineering

```python
import autogen
import os
from typing import Dict, List

# Configuración de LLMs
config_list = [
    {
        'model': 'gpt-4o',
        'api_key': os.getenv('OPENAI_API_KEY'),
        'temperature': 0
    }
]

llm_config = {
    'config_list': config_list,
    'timeout': 120,
    'cache_seed': 42  # Cache para reducir costos
}

# 1. DATA ENGINEER: Diseña pipelines y escribe código
data_engineer = autogen.AssistantAgent(
    name='DataEngineer',
    llm_config=llm_config,
    system_message="""Eres un Senior Data Engineer experto en:
- Diseño de pipelines ETL/ELT escalables
- SQL avanzado (Snowflake, BigQuery, Redshift)
- Python para data engineering (Pandas, Polars, PySpark)
- Airflow, dbt, Prefect para orquestación
- Best practices: idempotencia, retry logic, monitoring

Tu trabajo es:
1. Diseñar arquitecturas de datos robustas
2. Escribir código limpio y bien documentado
3. Considerar edge cases y manejo de errores
4. Proponer soluciones escalables

Siempre incluye:
- Código ejecutable con docstrings
- Manejo de errores con try/except
- Logging detallado
- Comentarios explicativos"""
)

# 2. QA ENGINEER: Revisa código, identifica bugs, sugiere tests
qa_engineer = autogen.AssistantAgent(
    name='QAEngineer',
    llm_config=llm_config,
    system_message="""Eres un QA Engineer especializado en calidad de data pipelines.

Tu trabajo es:
1. Revisar código para identificar:
   - Bugs lógicos
   - Performance bottlenecks
   - Missing error handling
   - Security vulnerabilities
   - Code smells

2. Sugerir tests:
   - Unit tests con pytest
   - Integration tests con mocks
   - Data quality tests

3. Validar:
   - Idempotencia del pipeline
   - Manejo de duplicados
   - Retry logic correcto

Sé crítico pero constructivo. Prioriza por impacto."""
)

# 3. ARCHITECT: Revisa arquitectura y decisiones técnicas
architect = autogen.AssistantAgent(
    name='TechArchitect',
    llm_config=llm_config,
    system_message="""Eres un Data Architect senior.

Tu trabajo es:
1. Revisar decisiones arquitecturales:
   - Escalabilidad (¿funciona con 10x datos?)
   - Costos (optimización de recursos)
   - Mantenibilidad (¿fácil de mantener?)
   - Seguridad (PII, compliance)

2. Sugerir alternativas:
   - Batch vs Streaming
   - Full load vs Incremental
   - Herramientas apropiadas

3. Validar patrones:
   - Data modeling (star schema, etc.)
   - Partitioning strategy
   - SLAs y monitoring

Enfócate en decisiones de alto impacto."""
)

# 4. USER PROXY: Ejecuta código y representa al usuario
user_proxy = autogen.UserProxyAgent(
    name='UserProxy',
    human_input_mode='NEVER',  # Modo automático (sin input humano)
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get('content', '').rstrip().endswith('TERMINATE'),
    code_execution_config={
        'work_dir': 'autogen_workspace',
        'use_docker': False  # En producción: usar Docker para sandbox
    },
    system_message="""Eres el User Proxy. Ejecutas código propuesto y reportas resultados.
    
Cuando recibas código Python:
1. Ejecutarlo en el workspace
2. Reportar output, errores, o éxito
3. Si hay errores, compartir el traceback completo"""
)

# 5. GROUP CHAT: Orquesta la conversación

# Crear group chat con todos los agentes
groupchat = autogen.GroupChat(
    agents=[user_proxy, data_engineer, qa_engineer, architect],
    messages=[],
    max_round=20,  # Máximo 20 turnos
    speaker_selection_method='auto'  # LLM decide quién habla
)

# Manager del group chat
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)

# 6. INICIAR CONVERSACIÓN

task = """
Tarea: Diseñar un pipeline ETL que:

1. EXTRACT:
   - Leer datos de API REST de Stripe (pagos)
   - API endpoint: /v1/charges
   - Rate limit: 100 requests/segundo
   - Necesita paginación (1000 records por página)

2. TRANSFORM:
   - Parsear JSON response
   - Convertir timestamps a datetime
   - Calcular métricas: total_amount, fee_amount, net_amount
   - Detectar y filtrar duplicados por charge_id

3. LOAD:
   - Insertar en Snowflake tabla: dwh.stripe_charges
   - Estrategia: upsert (update si existe, insert si nuevo)
   - Particionado por: payment_date

Requerimientos:
- Debe correr en Airflow (DAG)
- Idempotente (re-ejecutable sin duplicar datos)
- Manejo robusto de errores con reintentos
- Logging completo para debugging
- Tests unitarios con pytest

Diseñen el pipeline, revisen el código, y validen la arquitectura.
"""

print("=" * 80)
print("INICIANDO COLABORACIÓN MULTI-AGENTE")
print("=" * 80)
print(f"\nTarea:\n{task}\n")
print("=" * 80)

# Iniciar conversación
user_proxy.initiate_chat(
    manager,
    message=task
)

print("\n" + "=" * 80)
print("CONVERSACIÓN COMPLETADA")
print("=" * 80)
```

### 📊 Salida Esperada (Ejemplo)

```
UserProxy: [Tarea del pipeline ETL...]

DataEngineer: Voy a diseñar el pipeline en 3 componentes:

```python
# stripe_etl.py
import requests
import pandas as pd
from datetime import datetime
import logging
from tenacity import retry, stop_after_attempt, wait_exponential

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def extract_stripe_charges(api_key: str, start_date: str) -> pd.DataFrame:
    """Extrae charges de Stripe API con paginación y retry."""
    all_charges = []
    has_more = True
    starting_after = None
    
    while has_more:
        params = {
            'limit': 100,
            'created[gte]': int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
        }
        if starting_after:
            params['starting_after'] = starting_after
        
        response = requests.get(
            'https://api.stripe.com/v1/charges',
            auth=(api_key, ''),
            params=params,
            timeout=30
        )
        response.raise_for_status()
        
        data = response.json()
        all_charges.extend(data['data'])
        
        has_more = data['has_more']
        if has_more:
            starting_after = data['data'][-1]['id']
        
        logger.info(f"Extracted {len(data['data'])} charges, total: {len(all_charges)}")
    
    return pd.DataFrame(all_charges)

def transform_charges(df: pd.DataFrame) -> pd.DataFrame:
    """Transforma y limpia los datos."""
    df['payment_date'] = pd.to_datetime(df['created'], unit='s')
    df['total_amount'] = df['amount'] / 100  # Stripe usa cents
    df['net_amount'] = df['amount'] - df.get('fee', 0)) / 100
    
    # Deduplicar
    df = df.drop_duplicates(subset=['id'], keep='last')
    
    logger.info(f"Transformed {len(df)} charges")
    return df

def load_to_snowflake(df: pd.DataFrame, conn, table: str):
    """Carga datos a Snowflake con upsert."""
    # Crear temp table
    temp_table = f"{table}_temp"
    df.to_sql(temp_table, conn, if_exists='replace', index=False)
    
    # MERGE (upsert)
    merge_sql = f"""
    MERGE INTO {table} target
    USING {temp_table} source
    ON target.id = source.id
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *
    """
    conn.execute(merge_sql)
    logger.info(f"Loaded {len(df)} charges to {table}")
```

QAEngineer: Revisión del código:

**Problemas encontrados:**
1. ❌ `extract_stripe_charges`: No maneja rate limit 429
2. ❌ `transform_charges`: Error en línea `df.get('fee', 0)` - debe ser `df['fee'].fillna(0)`
3. ❌ `load_to_snowflake`: Falta manejo de errores
4. ⚠️ No hay tests unitarios
5. ⚠️ Falta validación de schema

**Sugerencias:**
```python
# Agregar rate limit handling
import time
from requests.exceptions import HTTPError

@retry(...)
def extract_stripe_charges(...):
    try:
        response = requests.get(...)
        response.raise_for_status()
    except HTTPError as e:
        if e.response.status_code == 429:
            retry_after = int(e.response.headers.get('Retry-After', 60))
            logger.warning(f"Rate limited, waiting {retry_after}s")
            time.sleep(retry_after)
            raise  # Retry
        else:
            raise

# Tests con pytest
def test_transform_charges():
    sample_data = [{'id': '1', 'amount': 1000, 'created': 1234567890}]
    df = pd.DataFrame(sample_data)
    result = transform_charges(df)
    assert result['total_amount'].iloc[0] == 10.0
    assert 'payment_date' in result.columns
```

TechArchitect: Revisión arquitectural:

**✅ Bueno:**
- Uso de retry con exponential backoff
- Paginación implementada correctamente
- Upsert strategy apropiada

**⚠️ Consideraciones:**
1. **Escalabilidad**: Para millones de records, considerar:
   - Batch processing en chunks (no todo en memoria)
   - Usar Spark o Polars en vez de Pandas

2. **Costos**:
   - Stripe API es costosa (rate limits)
   - Implementar incremental load (solo últimas 24h)
   - Cachear resultados en S3 antes de Snowflake

3. **Monitoring**:
   - Agregar métricas: records processed, API latency, error rate
   - Alertas si ETL falla o demora >30min

4. **Partitioning**:
   - Snowflake: particionar por `DATE_TRUNC('month', payment_date)`
   - Mejora queries analíticas por fecha

**Sugerencia de arquitectura:**
```
Airflow DAG (daily @ 2am)
  ↓
Extract (incremental: last 24h) → S3 staging (Parquet)
  ↓
Transform (Spark/Polars) → Validación schema
  ↓
Load (Snowflake COPY INTO) → Upsert con MERGE
  ↓
Data Quality checks (dbt tests)
  ↓
Notify (Slack si errores)
```

DataEngineer: Actualizando el código con feedback...

[Código mejorado con todos los fixes...]

UserProxy: Ejecutando tests... ✅ All tests passed (5/5)

TERMINATE
```

### 🎯 Patrones Avanzados con AutoGen

#### 1. Sequential Chat (Pipeline en etapas)

```python
# Conversación secuencial: output de agent 1 → input de agent 2

# Agente 1: Genera SQL
sql_generator = autogen.AssistantAgent(
    name='SQLGenerator',
    llm_config=llm_config,
    system_message='Generas queries SQL optimizadas.'
)

# Agente 2: Optimiza SQL
sql_optimizer = autogen.AssistantAgent(
    name='SQLOptimizer',
    llm_config=llm_config,
    system_message='Optimizas queries SQL para mejor performance.'
)

# Pipeline
user_proxy.initiate_chats([
    {
        'recipient': sql_generator,
        'message': 'Genera query para calcular ventas por región',
        'max_turns': 2,
        'summary_method': 'last_msg'
    },
    {
        'recipient': sql_optimizer,
        'message': 'Optimiza esta query',  # Recibe output del anterior
        'max_turns': 2
    }
])
```

#### 2. Nested Chats (Sub-equipos)

```python
# Equipo principal delega a sub-equipo especializado

# Sub-equipo: Especialistas en testing
test_writer = autogen.AssistantAgent(name='TestWriter', ...)
test_reviewer = autogen.AssistantAgent(name='TestReviewer', ...)

test_team = autogen.GroupChat(
    agents=[test_writer, test_reviewer],
    messages=[],
    max_round=5
)

# Agente principal puede invocar al sub-equipo
data_engineer.register_nested_chats(
    trigger=lambda msg: 'write tests' in msg.get('content', '').lower(),
    chat_queue=[{
        'recipient': test_team,
        'message': 'Escriban tests unitarios para este código'
    }]
)
```

#### 3. Custom Speaker Selection

```python
def custom_speaker_selection(last_speaker, groupchat):
    """Lógica personalizada para seleccionar siguiente speaker."""
    messages = groupchat.messages
    
    # Si último mensaje tiene código Python, QA Engineer revisa
    if '```python' in messages[-1].get('content', ''):
        return qa_engineer
    
    # Si hay problemas de arquitectura, Architect opina
    if 'scalability' in messages[-1].get('content', '').lower():
        return architect
    
    # Por defecto, round-robin
    return None  # Auto-select

groupchat = autogen.GroupChat(
    agents=[...],
    messages=[],
    max_round=20,
    speaker_selection_method=custom_speaker_selection
)
```

### 📊 Comparación: Single Agent vs Multi-Agent

| Aspecto | Single Agent | Multi-Agent (AutoGen) |
|---------|--------------|----------------------|
| **Complejidad tareas** | Simple-Media | Alta |
| **Calidad output** | Buena | Excelente (peer review) |
| **Especialización** | Generalista | Especialistas por rol |
| **Tiempo ejecución** | Rápido (1-5 iteraciones) | Lento (10-20 iteraciones) |
| **Costo (API calls)** | Bajo ($0.01-$0.10) | Alto ($0.50-$2.00) |
| **Debugging** | Más fácil | Más complejo |
| **Casos de uso** | Queries simples, clasificación | Diseño arquitectural, code review |

### 💰 Optimización de Costos en Multi-Agent

```python
# 1. Usar modelos pequeños para roles secundarios
llm_config_cheap = {
    'config_list': [{'model': 'gpt-3.5-turbo', 'api_key': api_key}],
    'cache_seed': 42
}

qa_engineer = autogen.AssistantAgent(
    name='QAEngineer',
    llm_config=llm_config_cheap,  # Modelo barato para revisiones
    ...
)

# 2. Límite estricto de rounds
groupchat = autogen.GroupChat(
    agents=[...],
    max_round=10,  # Máximo 10 turnos (previene loops infinitos)
    ...
)

# 3. Caching agresivo
llm_config = {
    'config_list': config_list,
    'cache_seed': 42,  # Mismo seed = cachea respuestas idénticas
    'temperature': 0    # Determinístico = más cacheable
}

# 4. Termination temprana
user_proxy = autogen.UserProxyAgent(
    ...
    is_termination_msg=lambda x: (
        'TERMINATE' in x.get('content', '') or
        'approved' in x.get('content', '').lower()
    )
)
```

### 🚀 Casos de Uso Reales en Data Engineering

| Caso de Uso | Agentes Necesarios | Beneficio |
|-------------|-------------------|-----------|
| **Code review de PRs** | Engineer + QA + Architect | Peer review automático |
| **Incident response** | Investigator + Engineer + Architect | Root cause analysis |
| **Pipeline design** | Engineer + QA + DBA | Diseño robusto validado |
| **Query optimization** | Analyst + DBA + Performance Engineer | Optimización multi-perspectiva |
| **Schema migration** | DBA + Engineer + Architect | Validación de impacto |
| **Data quality investigation** | Analyst + Engineer + Domain Expert | Diagnóstico profundo |

### 💡 Mejores Prácticas

1. **Roles bien definidos**: Cada agente debe tener un rol claro y no solapado
2. **System messages detallados**: Instrucciones específicas con ejemplos
3. **Límites de seguridad**: max_rounds, timeouts, cost limits
4. **Logging exhaustivo**: Guardar toda la conversación para debugging
5. **Human checkpoints**: Aprobación humana antes de acciones críticas
6. **Testing con mocks**: Probar conversaciones sin llamar APIs reales
7. **Monitoreo de costos**: Track API usage por conversación
8. **Termination clara**: Condiciones explícitas para terminar chat

## 🏭 Agentes en Producción: Automatización Real de Data Engineering

Los agentes autónomos están revolucionando Data Engineering al automatizar tareas que antes requerían intervención humana experta: **debugging de pipelines**, **optimización de queries**, **incident response**, y **monitoreo proactivo**. Aquí exploramos implementaciones de producción reales.

### 🚨 Caso 1: Incident Response Agent (Auto-Healing Pipelines)

Un agente que detecta, diagnostica y resuelve fallas en pipelines de Airflow automáticamente.

```python
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import json
from datetime import datetime, timedelta
from typing import Dict, List
import requests

# Tools para incident response

@tool
def get_airflow_dag_failures(hours: int = 24) -> str:
    """
    Obtiene lista de DAGs que han fallado en las últimas N horas.
    
    Args:
        hours: ventana de tiempo para buscar fallas
    
    Returns:
        JSON con DAGs fallidos y detalles de error
    """
    # En producción: llamar a Airflow API
    airflow_url = "http://airflow:8080/api/v1"
    headers = {"Authorization": f"Bearer {os.getenv('AIRFLOW_TOKEN')}"}
    
    end_date = datetime.now()
    start_date = end_date - timedelta(hours=hours)
    
    # Simular respuesta
    failures = {
        'daily_sales_etl': {
            'dag_id': 'daily_sales_etl',
            'execution_date': '2024-10-30T23:00:00',
            'state': 'failed',
            'task_id': 'extract_api_data',
            'error': 'requests.exceptions.Timeout: HTTPSConnectionPool(host=\'api.example.com\', port=443): Read timed out. (read timeout=30)',
            'duration_seconds': 35,
            'try_number': 3,
            'log_url': 'http://airflow:8080/log?dag_id=daily_sales_etl&task_id=extract_api_data&execution_date=2024-10-30T23:00:00'
        },
        'hourly_user_events': {
            'dag_id': 'hourly_user_events',
            'execution_date': '2024-10-30T22:00:00',
            'state': 'failed',
            'task_id': 'load_to_snowflake',
            'error': 'snowflake.connector.errors.DatabaseError: 250001: Connection closed unexpectedly',
            'duration_seconds': 1245,
            'try_number': 1,
            'log_url': 'http://airflow:8080/log?dag_id=hourly_user_events&task_id=load_to_snowflake&execution_date=2024-10-30T22:00:00'
        }
    }
    
    return json.dumps(failures, indent=2)

@tool
def get_task_logs(dag_id: str, task_id: str, execution_date: str) -> str:
    """
    Obtiene logs detallados de una tarea fallida.
    
    Args:
        dag_id: ID del DAG
        task_id: ID de la tarea
        execution_date: fecha de ejecución (ISO format)
    
    Returns:
        Logs completos de la tarea
    """
    # En producción: obtener de Airflow logs en S3/GCS
    sample_logs = {
        'daily_sales_etl/extract_api_data': """
[2024-10-30 23:00:15] INFO - Starting task extract_api_data
[2024-10-30 23:00:16] INFO - Calling API: https://api.example.com/sales
[2024-10-30 23:00:16] DEBUG - Request headers: {'Authorization': 'Bearer ***', 'Content-Type': 'application/json'}
[2024-10-30 23:00:16] DEBUG - Request params: {'start_date': '2024-10-30', 'limit': 1000}
[2024-10-30 23:00:46] ERROR - requests.exceptions.Timeout: HTTPSConnectionPool(host='api.example.com', port=443): Read timed out. (read timeout=30)
[2024-10-30 23:00:46] INFO - Retry 1/3 in 60 seconds...
[2024-10-30 23:01:46] ERROR - Timeout again after retry
[2024-10-30 23:01:46] ERROR - Task failed after 3 attempts
""",
        'hourly_user_events/load_to_snowflake': """
[2024-10-30 22:00:05] INFO - Loading 125000 records to Snowflake
[2024-10-30 22:00:05] INFO - Connection established to account: xyz123.us-east-1
[2024-10-30 22:00:10] INFO - Executing COPY INTO dwh.user_events FROM @stage/events_2024103022.parquet
[2024-10-30 22:15:30] ERROR - snowflake.connector.errors.DatabaseError: 250001: Connection closed unexpectedly
[2024-10-30 22:15:30] DEBUG - Network trace: Connection reset by peer (errno 104)
[2024-10-30 22:15:30] ERROR - Loaded 45230/125000 records before failure (36%)
"""
    }
    
    key = f"{dag_id}/{task_id}"
    return sample_logs.get(key, "Logs not found")

@tool
def check_external_service_status(service: str) -> str:
    """
    Verifica estado de servicios externos (APIs, databases).
    
    Args:
        service: nombre del servicio (api.example.com, snowflake, etc.)
    
    Returns:
        Estado del servicio y latencia
    """
    # En producción: healthcheck real o status page
    status_map = {
        'api.example.com': {
            'status': 'degraded',
            'latency_ms': 15234,
            'error_rate': 0.25,
            'message': 'API experiencing high latency due to traffic spike'
        },
        'snowflake': {
            'status': 'operational',
            'latency_ms': 234,
            'error_rate': 0.01,
            'message': 'All systems operational'
        }
    }
    
    return json.dumps(status_map.get(service, {'status': 'unknown'}), indent=2)

@tool
def retry_airflow_task(dag_id: str, task_id: str, execution_date: str) -> str:
    """
    Re-ejecuta una tarea fallida en Airflow.
    
    Args:
        dag_id: ID del DAG
        task_id: ID de la tarea
        execution_date: fecha de ejecución
    
    Returns:
        Resultado de la re-ejecución
    """
    # En producción: POST a Airflow API /dags/{dag_id}/dagRuns/{dag_run_id}/taskInstances/{task_id}/clear
    
    print(f"🔄 Retrying {dag_id}/{task_id}...")
    
    # Simular retry
    import time
    time.sleep(2)
    
    return json.dumps({
        'status': 'success',
        'message': f'Task {task_id} queued for retry',
        'new_try_number': 4,
        'estimated_start': '2024-10-31T00:05:00'
    })

@tool
def create_incident_ticket(title: str, description: str, severity: str) -> str:
    """
    Crea ticket de incidente en Jira/PagerDuty.
    
    Args:
        title: título del incidente
        description: descripción detallada
        severity: low, medium, high, critical
    
    Returns:
        ID del ticket creado
    """
    # En producción: integrar con Jira/PagerDuty API
    
    ticket = {
        'id': 'INC-12345',
        'title': title,
        'description': description,
        'severity': severity,
        'status': 'open',
        'created_at': datetime.now().isoformat(),
        'assigned_to': 'data-eng-oncall',
        'url': 'https://jira.company.com/browse/INC-12345'
    }
    
    print(f"🎫 Ticket creado: {ticket['id']}")
    return json.dumps(ticket, indent=2)

@tool
def send_slack_alert(channel: str, message: str, severity: str = 'info') -> str:
    """
    Envía alerta a Slack.
    
    Args:
        channel: canal de Slack (#data-engineering)
        message: mensaje a enviar
        severity: info, warning, error, critical
    
    Returns:
        Confirmación de envío
    """
    # En producción: Slack API
    
    emoji_map = {
        'info': ':information_source:',
        'warning': ':warning:',
        'error': ':x:',
        'critical': ':rotating_light:'
    }
    
    slack_message = f"{emoji_map.get(severity, ':bell:')} {message}"
    
    print(f"📢 Slack alert sent to {channel}: {slack_message[:100]}...")
    
    return json.dumps({
        'status': 'sent',
        'channel': channel,
        'timestamp': datetime.now().isoformat()
    })

# Crear agente de incident response

incident_tools = [
    get_airflow_dag_failures,
    get_task_logs,
    check_external_service_status,
    retry_airflow_task,
    create_incident_ticket,
    send_slack_alert
]

llm = ChatOpenAI(model='gpt-4o', temperature=0)

incident_prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un SRE especializado en incident response para data pipelines.

Tu proceso cuando detectas un incidente:

1. INVESTIGAR:
   - Obtener lista de DAGs fallidos
   - Revisar logs detallados
   - Verificar estado de servicios externos

2. DIAGNOSTICAR:
   - Identificar root cause (timeout, connection issue, data problem, etc.)
   - Clasificar severidad (low/medium/high/critical)

3. REMEDIAR:
   - Si es transient error → retry automático
   - Si es service degradation → alertar y escalar
   - Si es data issue → crear ticket para investigación

4. COMUNICAR:
   - Enviar alertas a Slack
   - Crear ticket si requiere investigación humana
   - Documentar acciones tomadas

IMPORTANTE:
- Solo retry automático si error es claramente transient (timeout, network)
- Para errores de datos o lógica, NO retry (crear ticket)
- Siempre alertar en Slack antes de acciones
- Priorizar minimizar downtime"""),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

incident_agent = create_openai_tools_agent(llm, incident_tools, incident_prompt)
incident_executor = AgentExecutor(
    agent=incident_agent,
    tools=incident_tools,
    verbose=True,
    max_iterations=15,
    handle_parsing_errors=True
)

# EJECUTAR: Incident response automático

print("=" * 80)
print("INCIDENT RESPONSE AGENT - AUTO-HEALING PIPELINES")
print("=" * 80)

result = incident_executor.invoke({
    "input": """Revisa los pipelines de las últimas 24 horas.
    Para cada falla:
    1. Diagnostica el problema
    2. Decide si es auto-recoverable
    3. Toma acción apropiada (retry o escalar)
    4. Alerta al equipo"""
})

print("\n" + "=" * 80)
print("RESUMEN DE ACCIONES")
print("=" * 80)
print(result['output'])
```

### Salida Esperada

```
> Entering new AgentExecutor chain...

Thought: Necesito revisar las fallas de las últimas 24h

Action: get_airflow_dag_failures
Action Input: {"hours": 24}

Observation: {
  "daily_sales_etl": {
    "error": "Timeout: Read timed out (read timeout=30)",
    ...
  },
  "hourly_user_events": {
    "error": "DatabaseError: Connection closed unexpectedly",
    ...
  }
}

Thought: Hay 2 DAGs fallidos. Voy a investigar cada uno.

Action: get_task_logs
Action Input: {"dag_id": "daily_sales_etl", "task_id": "extract_api_data", ...}

Observation: [Logs showing timeout after 30 seconds]

Action: check_external_service_status
Action Input: {"service": "api.example.com"}

Observation: {
  "status": "degraded",
  "latency_ms": 15234,
  "message": "API experiencing high latency due to traffic spike"
}

Thought: daily_sales_etl falló por API degraded (latencia 15s, timeout 30s).
Es un error transient - la API está operacional pero lenta.
Decisión: Retry con timeout aumentado.

Action: send_slack_alert
Action Input: {
  "channel": "#data-engineering",
  "message": "🔄 daily_sales_etl failed due to API timeout. API is degraded (15s latency). Retrying with increased timeout...",
  "severity": "warning"
}

Action: retry_airflow_task
Action Input: {"dag_id": "daily_sales_etl", "task_id": "extract_api_data", ...}

Observation: {
  "status": "success",
  "message": "Task queued for retry",
  "new_try_number": 4
}

Thought: Retry iniciado para daily_sales_etl. Ahora investigar hourly_user_events.

Action: get_task_logs
Action Input: {"dag_id": "hourly_user_events", "task_id": "load_to_snowflake", ...}

Observation: [Logs showing connection reset after loading 36% of data]

Action: check_external_service_status
Action Input: {"service": "snowflake"}

Observation: {
  "status": "operational",
  "message": "All systems operational"
}

Thought: hourly_user_events falló por connection reset durante COPY INTO.
Snowflake está operational, posiblemente network issue transient.
Pero solo 36% de data se cargó - puede haber duplicados si retry simple.
Decisión: Crear ticket para investigación manual (requiere análisis de datos parciales).

Action: create_incident_ticket
Action Input: {
  "title": "hourly_user_events failed mid-load to Snowflake",
  "description": "Task load_to_snowflake failed after loading 36% (45K/125K records). Connection reset by peer. Requires manual investigation to handle partial load and avoid duplicates.",
  "severity": "high"
}

Observation: {"id": "INC-12345", ...}

Action: send_slack_alert
Action Input: {
  "channel": "#data-engineering",
  "message": "🎫 hourly_user_events failed with partial data load. Created INC-12345 for investigation. Manual intervention required to resume safely.",
  "severity": "error"
}

Final Answer:
Revisé 2 pipelines fallidos en las últimas 24h:

1. ✅ daily_sales_etl (RECUPERADO AUTOMÁTICAMENTE)
   - Root cause: API timeout (servicio degradado, 15s latency)
   - Acción: Retry automático con timeout aumentado
   - Estado: En cola para re-ejecución

2. 🎫 hourly_user_events (REQUIERE INVESTIGACIÓN)
   - Root cause: Connection reset durante carga (network issue transient)
   - Problema: Carga parcial (36%) puede causar duplicados
   - Acción: Ticket INC-12345 creado para investigación manual
   - Estado: Esperando intervención del equipo

Alertas enviadas a #data-engineering para ambos casos.
```

### 📊 Caso 2: Query Optimization Agent

Agente que analiza queries lentas y las optimiza automáticamente.

```python
@tool
def get_slow_queries(minutes: int = 60, min_duration_seconds: float = 30.0) -> str:
    """Obtiene queries que han tardado más de N segundos."""
    # En producción: query a Snowflake QUERY_HISTORY o PostgreSQL pg_stat_statements
    slow_queries = [
        {
            'query_id': 'q_12345',
            'query_text': 'SELECT * FROM sales s JOIN customers c ON s.customer_id = c.id WHERE s.sale_date >= CURRENT_DATE - 30',
            'duration_seconds': 125.4,
            'rows_scanned': 150_000_000,
            'rows_returned': 1_250_000,
            'execution_time': '2024-10-30 23:45:12',
            'user': 'analytics_team'
        }
    ]
    return json.dumps(slow_queries, indent=2)

@tool
def explain_query(query: str) -> str:
    """Obtiene EXPLAIN ANALYZE del query."""
    # Simular execution plan
    explain = {
        'estimated_rows': 150_000_000,
        'actual_rows': 1_250_000,
        'estimated_cost': 25678.90,
        'execution_time_ms': 125400,
        'operations': [
            {
                'step': 1,
                'operation': 'Seq Scan on sales',
                'cost': 20000.00,
                'rows': 150_000_000,
                'warning': 'Full table scan - no index used'
            },
            {
                'step': 2,
                'operation': 'Hash Join',
                'cost': 5678.90,
                'rows': 1_250_000
            }
        ],
        'recommendations': [
            'Add index on sales(sale_date) for date filter',
            'Replace SELECT * with specific columns',
            'Consider partitioning sales table by sale_date'
        ]
    }
    return json.dumps(explain, indent=2)

@tool
def rewrite_query_optimized(original_query: str, optimizations: List[str]) -> str:
    """Reescribe query aplicando optimizaciones."""
    # LLM generaría esto, aquí simulamos
    optimized = """
-- OPTIMIZED VERSION:
-- Changes: 1) Added index hint, 2) Specific columns, 3) Materialized subquery

SELECT 
    s.sale_id,
    s.sale_date,
    s.total_amount,
    c.customer_name,
    c.customer_email
FROM sales s
USE INDEX (idx_sales_date)  -- Hint to use index
INNER JOIN customers c ON s.customer_id = c.id
WHERE s.sale_date >= CURRENT_DATE - 30
  AND s.total_amount > 0  -- Additional filter to reduce rows

-- Expected improvement: 95% faster (125s → 6s)
"""
    return optimized

# Uso
optimizer_tools = [get_slow_queries, explain_query, rewrite_query_optimized]
# [Configurar agente similar...]
```

### 🎯 Caso 3: Data Quality Monitoring Agent

```python
@tool
def run_data_quality_checks(table: str) -> str:
    """Ejecuta suite completa de data quality checks."""
    checks = {
        'nulls': {'column': 'email', 'null_rate': 0.156, 'threshold': 0.05, 'status': 'FAIL'},
        'duplicates': {'count': 450, 'percentage': 0.009, 'threshold': 0.01, 'status': 'PASS'},
        'freshness': {'last_update': '2024-10-30 20:30:00', 'delay_hours': 3.5, 'threshold': 2.0, 'status': 'FAIL'},
        'schema': {'columns_expected': 12, 'columns_actual': 12, 'status': 'PASS'},
        'referential_integrity': {'orphaned_records': 23, 'threshold': 0, 'status': 'FAIL'}
    }
    return json.dumps(checks, indent=2)

@tool
def investigate_data_anomaly(table: str, column: str, anomaly_type: str) -> str:
    """Investiga causa raíz de anomalía en datos."""
    # Análisis con LLM + queries SQL
    investigation = {
        'anomaly': f'{anomaly_type} in {table}.{column}',
        'potential_causes': [
            'Upstream pipeline failure',
            'Schema change in source',
            'Data validation disabled'
        ],
        'affected_rows': 15600,
        'first_occurrence': '2024-10-28 14:23:00',
        'recommendation': 'Check upstream pipeline and re-run with validation'
    }
    return json.dumps(investigation, indent=2)

# El agente detecta fallas de calidad, investiga y alerta automáticamente
```

### 📈 Métricas de Éxito de Agentes en Producción

| Métrica | Target | Medición |
|---------|--------|----------|
| **MTTR (Mean Time To Recovery)** | <15 min | Tiempo desde falla hasta resolución |
| **Auto-resolution rate** | >60% | % de incidents resueltos sin humano |
| **False positive rate** | <5% | % de alertas/acciones incorrectas |
| **Cost per incident** | <$0.50 | Costo en API calls por incident |
| **Precision de diagnóstico** | >90% | % de root causes identificadas correctamente |
| **Uptime improvement** | +15% | Reducción de downtime vs manual |

### 💰 ROI de Agentes Autónomos

```python
# Cálculo de ROI

# Sin agentes (manual)
incidents_per_month = 45
avg_resolution_time_hours = 2.0
engineer_hourly_rate = 75  # USD
manual_cost = incidents_per_month * avg_resolution_time_hours * engineer_hourly_rate
# = 45 * 2 * $75 = $6,750/mes

# Con agentes
auto_resolved = incidents_per_month * 0.60  # 60% auto-resolved
manual_remaining = incidents_per_month * 0.40
avg_resolution_time_with_agent = 0.5  # hours (más rápido con diagnóstico automático)

manual_cost_with_agent = manual_remaining * avg_resolution_time_with_agent * engineer_hourly_rate
# = 18 * 0.5 * $75 = $675/mes

api_costs = incidents_per_month * 0.25  # $0.25 por incident en API calls
# = 45 * $0.25 = $11.25/mes

total_cost_with_agent = manual_cost_with_agent + api_costs
# = $675 + $11.25 = $686.25/mes

savings = manual_cost - total_cost_with_agent
# = $6,750 - $686 = $6,064/mes = $72,768/año

roi = (savings / total_cost_with_agent) * 100
# = 883% ROI
```

### 🚧 Desafíos y Limitaciones

| Desafío | Mitigación |
|---------|-----------|
| **Alucinaciones** | Validación de outputs, dry-run mode |
| **Costos API** | Caching, modelos pequeños, rate limiting |
| **Latencia** | Async execution, timeouts, parallelización |
| **Confiabilidad** | Retry logic, fallback a manual, human approval gates |
| **Security** | Tool sandboxing, permission checks, audit logs |
| **Debugging** | Exhaustive logging, replay capability |

### 🎯 Roadmap para Implementar Agentes

**Fase 1: Observación (Mes 1-2)**
- Agente solo observa y reporta (no toma acciones)
- Logging de decisiones que tomaría
- Validación de precisión de diagnósticos

**Fase 2: Asistencia (Mes 3-4)**
- Agente sugiere acciones, humano aprueba
- Human-in-the-loop para todas las decisiones
- Medir false positive rate

**Fase 3: Semi-autonomía (Mes 5-6)**
- Auto-resolución de incidents "seguros" (read-only actions, retries)
- Acciones destructivas requieren aprobación
- Monitoreo 24/7 de métricas

**Fase 4: Autonomía completa (Mes 7+)**
- Auto-resolución de 60-80% de incidents
- Alertas solo para casos complejos
- Continuous learning con feedback

### 💡 Mejores Prácticas para Agentes de Producción

1. **Start small**: Comenzar con un caso de uso simple (ej. retry de timeouts)
2. **Dry-run mode**: Implementar modo de prueba que no ejecuta acciones reales
3. **Approval gates**: Requerir aprobación humana para acciones críticas
4. **Exhaustive logging**: Loggear TODO (inputs, outputs, decisiones, acciones)
5. **Rollback capability**: Poder deshacer acciones del agente
6. **Cost monitoring**: Alertar si costo por incident excede threshold
7. **A/B testing**: Comparar agente vs manual con métricas objetivas
8. **Feedback loop**: Permitir humanos marcar decisiones como correctas/incorrectas
9. **Gradual rollout**: 10% → 50% → 100% de incidents
10. **Emergency stop**: Botón para desactivar agente si se comporta mal

### 🚀 Futuro: Agentes Cognitivos

```
2024: Rule-based + LLM reasoning
  ↓
2025: Learning from outcomes (RL)
  ↓
2026: Proactive optimization (no espera fallas)
  ↓
2027: Self-improving pipelines
  ↓
2030: Fully autonomous data platforms
```

El futuro de Data Engineering es **agentic**: sistemas que no solo ejecutan instrucciones, sino que **razonan, aprenden y se adaptan** continuamente para mantener pipelines de datos operando con mínima intervención humana.

## 1. Agente simple con LangChain

In [ ]:
# pip install langchain langgraph openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain import hub

llm = ChatOpenAI(model='gpt-4', temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'))

@tool
def get_table_row_count(table_name: str) -> str:
    """Devuelve el número de filas de una tabla."""
    # Simulación
    counts = {'ventas': 1250000, 'clientes': 50000, 'productos': 8500}
    return f'La tabla {table_name} tiene {counts.get(table_name, 0)} filas.'

@tool
def get_table_schema(table_name: str) -> str:
    """Devuelve el esquema de una tabla."""
    schemas = {
        'ventas': 'venta_id (BIGINT), fecha (DATE), producto_id (INT), total (DECIMAL)',
        'clientes': 'cliente_id (INT), nombre (VARCHAR), email (VARCHAR), ciudad (VARCHAR)'
    }
    return schemas.get(table_name, 'Tabla no encontrada')

tools = [get_table_row_count, get_table_schema]

# Prompt
prompt = hub.pull('hwchase17/openai-tools-agent')

# Crear agente
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({
    'input': '¿Cuántas filas tiene la tabla ventas y cuál es su esquema?'
})

print('\nRespuesta final:', result['output'])

## 2. Agente para debugging de SQL

In [ ]:
@tool
def validate_sql_syntax(query: str) -> str:
    """Valida sintaxis SQL."""
    # Simulación simple
    errors = []
    if 'SELCT' in query.upper():
        errors.append('Typo: SELCT debería ser SELECT')
    if query.count('(') != query.count(')'):
        errors.append('Paréntesis desbalanceados')
    return 'Errores: ' + ', '.join(errors) if errors else 'Sintaxis válida'

@tool
def suggest_sql_optimization(query: str) -> str:
    """Sugiere optimizaciones."""
    tips = []
    if 'SELECT *' in query.upper():
        tips.append('Evita SELECT *, especifica columnas')
    if 'WHERE' not in query.upper() and 'JOIN' in query.upper():
        tips.append('Considera agregar WHERE para filtrar resultados')
    return 'Sugerencias: ' + ', '.join(tips) if tips else 'Query está bien optimizada'

debug_tools = [validate_sql_syntax, suggest_sql_optimization]

debug_agent = create_openai_tools_agent(llm, debug_tools, prompt)
debug_executor = AgentExecutor(agent=debug_agent, tools=debug_tools, verbose=True)

broken_query = 'SELCT * FROM ventas JOIN productos'

result = debug_executor.invoke({
    'input': f'Analiza esta query y sugiere mejoras: {broken_query}'
})

## 3. LangGraph: workflow con estados

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

class ETLState(TypedDict):
    query: str
    validated: bool
    optimized: bool
    result: str
    errors: Annotated[list, operator.add]

def validate_node(state: ETLState) -> ETLState:
    """Nodo de validación."""
    if 'SELECT' not in state['query'].upper():
        state['errors'].append('Query no es SELECT')
        state['validated'] = False
    else:
        state['validated'] = True
    return state

def optimize_node(state: ETLState) -> ETLState:
    """Nodo de optimización."""
    if state['validated']:
        # Simulación
        state['query'] = state['query'].replace('SELECT *', 'SELECT id, nombre')
        state['optimized'] = True
    return state

def execute_node(state: ETLState) -> ETLState:
    """Nodo de ejecución."""
    if state['optimized']:
        state['result'] = f'Query ejecutada: {state["query"]}'
    else:
        state['result'] = 'Ejecución cancelada por errores'
    return state

# Crear grafo
workflow = StateGraph(ETLState)
workflow.add_node('validate', validate_node)
workflow.add_node('optimize', optimize_node)
workflow.add_node('execute', execute_node)

workflow.set_entry_point('validate')
workflow.add_edge('validate', 'optimize')
workflow.add_edge('optimize', 'execute')
workflow.add_edge('execute', END)

app = workflow.compile()

# Ejecutar
initial_state = {
    'query': 'SELECT * FROM ventas',
    'validated': False,
    'optimized': False,
    'result': '',
    'errors': []
}

final_state = app.invoke(initial_state)
print('Estado final:', final_state)

## 4. AutoGen: múltiples agentes colaborativos

In [ ]:
# pip install pyautogen
import autogen

config_list = [{
    'model': 'gpt-4',
    'api_key': os.getenv('OPENAI_API_KEY')
}]

# Agente 1: Data Engineer
data_engineer = autogen.AssistantAgent(
    name='DataEngineer',
    llm_config={'config_list': config_list},
    system_message='''
Eres un ingeniero de datos experto. Tu tarea es:
1. Diseñar pipelines ETL
2. Escribir queries SQL optimizadas
3. Proponer arquitecturas de datos
'''
)

# Agente 2: QA Engineer
qa_engineer = autogen.AssistantAgent(
    name='QAEngineer',
    llm_config={'config_list': config_list},
    system_message='''
Eres un ingeniero de calidad. Tu tarea es:
1. Revisar código y queries
2. Identificar bugs y problemas de rendimiento
3. Sugerir tests
'''
)

# Agente 3: User Proxy (humano)
user_proxy = autogen.UserProxyAgent(
    name='User',
    human_input_mode='NEVER',
    max_consecutive_auto_reply=0
)

# Conversación
task = '''
Diseña un pipeline que:
1. Extraiga datos de API de ventas
2. Transforme (agregue por día)
3. Cargue en Redshift
Luego revisa el diseño.
'''

# Iniciar chat grupal
groupchat = autogen.GroupChat(
    agents=[user_proxy, data_engineer, qa_engineer],
    messages=[],
    max_round=5
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={'config_list': config_list})

user_proxy.initiate_chat(manager, message=task)

## 5. Agente para monitoreo de data quality

In [ ]:
@tool
def check_null_rate(table: str, column: str) -> str:
    """Verifica tasa de nulos."""
    # Simulación
    rates = {('ventas', 'total'): 0.5, ('clientes', 'email'): 15.2}
    rate = rates.get((table, column), 0)
    return f'Tasa de nulos en {table}.{column}: {rate}%'

@tool
def check_duplicates(table: str) -> str:
    """Verifica duplicados."""
    dup_counts = {'ventas': 120, 'clientes': 5}
    count = dup_counts.get(table, 0)
    return f'Duplicados en {table}: {count}'

quality_tools = [check_null_rate, check_duplicates]

quality_agent = create_openai_tools_agent(llm, quality_tools, prompt)
quality_executor = AgentExecutor(agent=quality_agent, tools=quality_tools, verbose=True)

quality_result = quality_executor.invoke({
    'input': 'Revisa la calidad de la tabla ventas: nulos y duplicados'
})

## 6. Agente reactivo con memoria

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

agent_with_memory = create_openai_tools_agent(llm, tools, prompt)
executor_with_memory = AgentExecutor(
    agent=agent_with_memory,
    tools=tools,
    memory=memory,
    verbose=True
)

# Conversación
executor_with_memory.invoke({'input': 'Cuántas filas tiene ventas?'})
executor_with_memory.invoke({'input': 'Y cuál es su esquema?'})  # Contexto previo

## 7. Buenas prácticas

- **Tools específicas**: cada tool debe hacer UNA cosa bien.
- **Validación de outputs**: verifica respuestas antes de acciones críticas.
- **Human-in-the-loop**: aprobación humana para operaciones destructivas.
- **Observabilidad**: loggea cada decisión del agente.
- **Fallback**: maneja errores de LLM o tools.
- **Testing**: prueba agentes con casos edge.
- **Costos**: limita llamadas y usa modelos pequeños cuando posible.

## 8. Ejercicios

1. Crea un agente que automatice la investigación de incidentes en pipelines.
2. Construye un agente que genere data quality reports diarios.
3. Implementa un sistema multi-agente para code review de PRs.
4. Desarrolla un agente que optimice queries en producción automáticamente.

---

## 🧭 Navegación

**← Anterior:** [🔍 Embeddings y Similitud en Datos](05_embeddings_similitud_datos.ipynb)

**Siguiente →:** [✅ Validación de Datos con LLMs →](07_calidad_validacion_llm.ipynb)

**📚 Índice de Nivel GenAI:**
- [🔄 Comparación: OpenAI vs Google Gemini](00_comparacion_openai_gemini.ipynb)
- [🤖 Fundamentos de LLMs y Prompting](01_fundamentos_llms_prompting.ipynb)
- [📊 Text-to-SQL: Generación de Consultas SQL desde Lenguaje Natural](02_generacion_sql_nl2sql.ipynb)
- [🔧 Generación Automática de Código ETL con LLMs](03_generacion_codigo_etl.ipynb)
- [📚 RAG: Documentación Técnica con LLMs](04_rag_documentacion_datos.ipynb)
- [🔍 Embeddings y Similitud en Datos](05_embeddings_similitud_datos.ipynb)
- [🤖 Agentes Autónomos para Automatización](06_agentes_automatizacion.ipynb) ← 🔵 Estás aquí
- [✅ Validación de Datos con LLMs](07_calidad_validacion_llm.ipynb)
- [🎲 Generación de Datos Sintéticos con LLMs](08_sintesis_aumento_datos.ipynb)
- [🚀 Proyecto Integrador 1: Chatbot de Consulta de Datos con RAG](09_proyecto_integrador_1.ipynb)
- [🏗️ Proyecto Integrador 2: Plataforma Self-Service con GenAI](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
